In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
# READ AND DROP COLUMNS
df1 = pd.read_csv("D:/DM_Project_Dataset/training_cleaned.csv")
df = df1.loc[:, ['Nationality',
 'TopThreeAmericanName',
 'Make',
 'RetailAVG',
 'AuctionAVG',
 'VehBCost']]
df.head()

,Nationality,TopThreeAmericanName,Make,RetailAVG,AuctionAVG,VehBCost
0,OTHER ASIAN,OTHER,KIA,10300.0,7128.5,6100.0
1,AMERICAN,CHRYSLER,DODGE,6948.0,5970.5,4000.0
2,AMERICAN,CHRYSLER,DODGE,11199.0,8061.5,7500.0
3,AMERICAN,FORD,FORD,6696.5,5737.5,4725.0
4,AMERICAN,CHRYSLER,CHRYSLER,7046.0,6061.0,5670.0


## Normalisation and Encoding

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Identify the string columns
string_cols = df.select_dtypes(include='object').columns.tolist()

# Loop over each string column and encode it using a LabelEncoder
for col in string_cols:
    if col in df.columns:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_norm = scaler.fit_transform(df)

In [4]:
y = df1['IsBadBuy']
X = df_norm

In [5]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [6]:
# Set the parameters by cross-validation
tuned_parameters = {
    'n_estimators': [25, 50, 100, 200, 500, 1000]
}

#scores = ['precision', 'recall', 'f1']
scores = ['recall', 'f1']

for score in scores:
    print("# Tuning hyper-parameters for ----> %s" % score)
    print()
    
    obj = RandomForestClassifier()
    
    if (score == "recall"):
      clf = GridSearchCV(obj, tuned_parameters, cv=5, scoring=score)
    else:
      clf = GridSearchCV(obj, tuned_parameters, cv=5, scoring='%s_macro' % score)

    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for ----> recall

Best parameters set found on development set:

{'n_estimators': 25}

Grid scores on development set:

0.038 (+/-0.013) for {'n_estimators': 25}
0.029 (+/-0.019) for {'n_estimators': 50}
0.028 (+/-0.016) for {'n_estimators': 100}
0.028 (+/-0.018) for {'n_estimators': 200}
0.028 (+/-0.016) for {'n_estimators': 500}
0.029 (+/-0.014) for {'n_estimators': 1000}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.88      0.98      0.93     20294
           1       0.23      0.04      0.07      2912

    accuracy                           0.86     23206
   macro avg       0.55      0.51      0.50     23206
weighted avg       0.80      0.86      0.82     23206


# Tuning hyper-parameters for ----> f1

Best parameters set found on development set:

{'n_estimators': 25}

Grid scores on developm